In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from mpl_toolkits.mplot3d import Axes3D

class RBFNeuralNetwork:
    '''simple Radial-Basis Function Neural Network class'''
    def __init__(self,num_inputs,num_outputs,num_neurons,tf_functions,epochs=100,learning_rate=0.1,mode='sequential'):
        self.mode = mode
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.num_neurons = num_neurons

        self.min_weight_value = -1
        self.max_weight_value = 1
        self.tf_functions = []
        #self.tf_functions_derivatives = []

        for i in range(0,len(tf_functions)):
            if tf_functions[i] == "gaussian":
                self.tf_functions.append(self.tf_gaussian)
                #self.tf_functions_derivatives.append(self.tf_logistic_derivative)
            elif tf_functions[i] == "linear":
                self.tf_functions.append(self.tf_linear)
            #elif tf_functions[i] == "tanh":
            #    self.tf_functions.append(self.tf_tanh)
                #self.tf_functions_derivatives.append(self.tf_linear_derivative)
            else:
                print("Unknown transfer function: %s" %(tf_function[i]))
                exit()
        self.outputs = None # will be updated after calculation is called

        #********************************
        # Initialize centers, weights and biases
        #********************************

        centers_per_layer = [] # for RBF
        weights_per_layer = []
        biases_per_layer = []
        wsums_per_layer = []  # for helping later activation function approximate derivative calculation

        for l in range(0,len(num_neurons)): # for all layers
            previous_layer_is_input = 0
            if (l == 0):
                previous_layer_size = len(inputs[0])
                previous_layer_is_input = 1
            else:
                previous_layer_size = num_neurons[l-1]
            layer_size = num_neurons[l]
            # needed only for RBF networks
            if (not previous_layer_is_input):
                layer_weights = np.zeros((layer_size,previous_layer_size))
                layer_biases = np.zeros(layer_size)
            else:  # if previous layer is input
                layer_weights = np.zeros((layer_size,0))
                #ipdb.set_trace()
                layer_biases = np.zeros((0))
                # Setting layer centers
                layer_centers = np.zeros((layer_size,previous_layer_size))
                #layer_centers = np.zeros((1,previous_layer_size))   # one set of centers per layer, the set with the same dimension of the previous layer
                self.centers_per_layer = layer_centers

            weights_per_layer.append(layer_weights)
            biases_per_layer.append(layer_biases)
            wsums_per_layer.append(np.zeros(layer_size))

        self.weights_per_layer = weights_per_layer
        self.biases_per_layer = biases_per_layer
        self.wsums_per_layer = wsums_per_layer

        self.randomize_weights()
        self.randomize_biases()
        self.randomize_centers()
        #self.weights_per_layer = [np.array([[0.5, 0.5],[0.5, 0.5]]),
        #                         np.array([[0.3,-0.5]])]



    def set_last_layer_weights(self,weights):
        self.weights_per_layer[-1] = [weights]

    def set_centers(self,centers):
        self.centers_per_layer = centers

    def set_weights_interpolation_matrix(self,inputs,outputs):
        interp_matrix = np.zeros((inputs.shape[0],self.centers_per_layer.shape[0]))
        for j in range(0,len(inputs)):
            for i in range(0,len(self.centers_per_layer)):
                phi = self.neuron_rbf(self.centers_per_layer[i],inputs[j],self.tf_functions[0],0,i)
                interp_matrix[j,i] = phi

        interp_matrix = np.array(interp_matrix)

        # w = inv(interp_matrix) * d
        W = np.linalg.pinv(interp_matrix).dot(outputs)
        self.weights_per_layer[1][0] = W


    def randomize_centers(self):
        a = self.min_weight_value
        b = self.max_weight_value
        for i, layer in enumerate(self.centers_per_layer):
            #self.weights_per_layer[i] = np.random.random(self.weights_per_layer[i].shape)
            self.centers_per_layer[i] = a+(b-a)*np.random.random(self.centers_per_layer[i].shape)
            #self.weights_per_layer[i] = self.weights_per_layer[i]*0 # temp, makes them all 0

    def randomize_weights(self):
        a = self.min_weight_value
        b = self.max_weight_value
        for i, layer in enumerate(self.weights_per_layer):
            #self.weights_per_layer[i] = np.random.random(self.weights_per_layer[i].shape)
            self.weights_per_layer[i] = a+(b-a)*np.random.random(self.weights_per_layer[i].shape)
            #self.weights_per_layer[i] = self.weights_per_layer[i]*0 # temp, makes them all 0

    def randomize_biases(self):
        a = self.min_weight_value
        b = self.max_weight_value
        for i, layer in enumerate(self.biases_per_layer):
            #self.biases_per_layer[i] = a+(b-a)*np.random.random(self.biases_per_layer[i].shape)
            self.biases_per_layer[i] = self.biases_per_layer[i]*0 # temp, makes them all 0

    def predict(self,X):
        Ypredicted = []
        for i in range(0,X.shape[0]):  #for every pattern
            predicted = model.calc_output(X[i]) # calculating outputs for a given pattern
            Ypredicted.append(predicted)
        return np.array(Ypredicted)

    def calc_output(self,inputs):
        '''calculates the output of the neural network'''
        #*****************
        # creates empty output array
        #*****************
        outputs = []
        for i in range(0,len(self.num_neurons)):
            outputs.append(np.zeros(num_neurons[i]))

        outputs = np.array(outputs) #convert to numpy array
        #*****************
        # calculates output for each layer
        #*****************
        for i in range(0,len(self.num_neurons)):
            #if (i == 0): # first layer
            #    outputs[i] = self.calc_layer(inputs,self.weights_per_layer[i],self.biases_per_layer[i],self.tf_functions[i],i)
            if (i == 0): # first layer
                outputs[i] = self.calc_layer_rbf(inputs,self.weights_per_layer[i],self.biases_per_layer[i],self.centers_per_layer,self.tf_functions[i],i)
            else:
                outputs[i] = self.calc_layer(outputs[i-1],self.weights_per_layer[i],self.biases_per_layer[i],self.tf_functions[i],i)
        self.outputs = outputs

        return outputs[-1]

    def calc_layer_rbf(self,inputs,weights,biases,centers,tf_function,layer_index):
        # weights and biases will not be used, except for knowing the number of outputs
        outputs = np.zeros(len(weights))
        for i in range(0,centers.shape[0]): #for all neurons
            # outputs[i] = self.neuron_rbf(centers,inputs,tf_function,layer_index,i) #same centers
            outputs[i] = self.neuron_rbf(centers[i],inputs,tf_function,layer_index,i)  # one center vector per neuron
        return outputs

    def neuron_rbf(self,centers,inputs,tf_function,layer_index,neuron_index):
        #weighted_sum = np.dot(weights,inputs)+bias
        summing_section = inputs-centers
        norm_section = np.linalg.norm(summing_section,ord=2)
        output = tf_function(norm_section,sigma=5)  # phi

        # update for later calculate activation function approximate derivative
        self.wsums_per_layer[layer_index][neuron_index] = norm_section

        return output


    def calc_layer(self,inputs,weights,biases,tf_function,layer_index):
        outputs = np.zeros(len(weights))
        for i in range(0,len(weights)): #for all neurons
            outputs[i] = self.neuron(weights[i],biases[i],inputs,tf_function,layer_index,i)
        return outputs

    def tf_gaussian(self,x,sigma=1):
        return np.exp(-((x)/sigma)**2)

    def tf_logistic(self,x):
        return 1/(1+np.exp(-x))
        #return 1/(1+np.exp(-x))

    def tf_tanh(self,x):
        a = 1.7159
        b = 2/3
        return a*np.tanh(b*x)



    def tf_logistic_derivative(self,y):
        return (1-y)*(y)  #https://en.wikipedia.org/wiki/Logistic_function#Derivative #https://sebastianraschka.com/faq/docs/logistic-sigmoid-derivative.html

    def tf_linear(self,x):
        return x
    def tf_linear_derivative(self,y):
        return 1

    def neuron(self,weights,bias,inputs,tf_function,layer_index,neuron_index):
        weighted_sum = np.dot(weights,inputs)+bias
        output = tf_function(weighted_sum)

        # update for later calculate activation function approximate derivative
        self.wsums_per_layer[layer_index][neuron_index] = weighted_sum

        return output


    def print_report(self,X,Y,epoch):
        #print("Inputs: ")
        # print("###################")
        print("## Epoch: %d" %epoch)
        for i,x in enumerate(X):
            print("in%d: %f " %(i,x),end="")
        print("")
        for l in range(0,len(self.weights_per_layer)): #for all layers
            #ipdb.set_trace()
            print("layer %d \t " %l)
            for nl in range(0,self.weights_per_layer[l].shape[0]): #for all going to neuron nl of this layer
                for np in range(0,self.weights_per_layer[l].shape[1]): #for all coming from neuron np of the previous layer
                    if (l == 0):
                        print("w_in%d_l%dn%d: %f \t" %(np,l,nl,self.weights_per_layer[l][nl,np]),end='')
                    else:
                        print("w_n%d_l%dn%d: %f \t" %(np,l,nl,self.weights_per_layer[l][nl,np]),end='')
                for bi in range(0,self.biases_per_layer[l].shape[0]):
                    print("b_l%dn%d: %f \t " %(l,bi,self.biases_per_layer[l][bi]),end='')
                print("\n")
                sum_ln = self.wsums_per_layer[l][nl]
                tf = self.tf_functions[l]
                deriv_sum_ln =self.num_derivative(tf,self.wsums_per_layer[l][nl])
                print("- sum_l%dn%d: %f" %(l,nl,sum_ln))
                print("- output_l%dn%d: %f" %(l,nl,self.outputs[l][nl]))
                print("- deriv_sum_l%dn%d: %f" %(l,nl,deriv_sum_ln))
                print("- local_gradient_l%dn%d: %f" %(l,nl,self.local_gradients[l][nl]))
                print("")
        for o in range(0,self.outputs[-1].shape[0]):
            error = Y-self.outputs[-1][o] #desired - output
            print("output: %f \t desired output: %f \t error: %f" %(self.outputs[-1][o],Y,error))
        print("")
        print("###################")


    def fit(self,X,Y):

        # sequential approach
        if (self.mode == "sequential"):
            self.ssr_total_list = []
            self.mse_total_list = []
            for epc in range(0,self.epochs+1):
                ssr_total = 0
                mse_total = 0
                idxlist = np.arange(0,X.shape[0])

                #on first evaluation, local gradients are not updated, just to save the initial solution
                #np.random.shuffle(idxlist)  #randomize index_list
                for i in range(0,X.shape[0]):  #for every pattern
                    predicted = model.calc_output(X[idxlist[i]]) # calculating outputs for a given pattern
                    output_error = Y[idxlist[i]] - predicted  # error for each output
                    ssr = 0.5*sum(output_error**2) #sum of squared residuals
                    ssr_total += ssr
                    mse_total += sum(output_error**2)
                    # calculate local gradients
                    #if (epc != 0):
                    self.calculate_local_gradients(output_error,ssr)
                    #self.print_report(X[i],Y[i],epc)
                    self.update_weights(X[idxlist[i]])

                mse = mse_total/X.shape[0]
                self.ssr_total_list.append(ssr_total)
                self.mse_total_list.append(mse)

        else: # (self.mode == "batch"): #not working properly
            self.ssr_total_list = []
            self.mse_total_list = []
            for epc in range(0,self.epochs+1):
                ssr_total = 0
                mse_total = 0
                idxlist = np.arange(0,X.shape[0])
                for i in range(0,X.shape[0]):  #for every pattern
                    predicted = model.calc_output(X[i]) # calculating outputs for a given pattern
                    output_error = Y[i] - predicted  # error for each output
                    ssr = 0.5*sum(output_error**2) #sum of squared residuals
                    ssr_total += ssr
                    mse_total += sum(output_error**2)
                    # calculate local gradients
                    if (epc != 0):
                        self.calculate_local_gradients(output_error,ssr)
                        if (i==0):
                            shapes = [arr.shape for arr in self.local_gradients]
                            dw = [np.zeros(shapes[0])]
                            for k in range(1,len(shapes)):
                                dw.append(np.zeros(shapes[k]))
                        for j in range(0, len(dw)):
                            dw[j] += self.local_gradients[j]/X.shape[0]

                #update all weights
                if (epc != 0):
                    self.local_gradients = dw[:] #update total gradient
                    for i in range(0,X.shape[0]):  #for every pattern
                        self.update_weights(X[i])
                mse = mse_total/X.shape[0]
                self.ssr_total_list.append(ssr_total)
                self.mse_total_list.append(mse)





    def calculate_local_gradients(self,output_error,ssr):
        ''' calculates local gradients '''
        # initializes local gradients
        self.local_gradients = [0]*len(num_neurons) # will be a list for each element
        for i in range(0,len(self.local_gradients)):
            self.local_gradients[i] = np.zeros(num_neurons[i])

        # calculates local gradients for output layer
        for j in range(0,self.num_neurons[-1]): # for all neurons in the output layer
            partiald_E_y = -(output_error[j])
            #local_gradient = -partiald_E_y * self.tf_functions_derivatives[-1](self.outputs[-1][j])
            tf = self.tf_functions[-1]
            local_gradient = -partiald_E_y * 1
            self.local_gradients[-1][j] = local_gradient

        # calculates local gradients for hidden layers
        for l in range(len(num_neurons)-2,-1,-1):  # for all hidden layers, from last to first
            for j in range(0,num_neurons[l]):
                outsum = 0
                w_from_this_neuron_to_next_layer = self.weights_per_layer[l+1][:,j]
                for o in range(0,num_neurons[l+1]): # for all neurons on the next layer
                    wok = w_from_this_neuron_to_next_layer[o]
                    outsum+= self.local_gradients[l+1][o]*wok
                #self.local_gradients[l][j] = self.tf_functions_derivatives[l](self.outputs[l][j])*outsum
                tf = self.tf_functions[l]
                self.local_gradients[l][j] = self.num_derivative(tf,self.wsums_per_layer[l][j])*outsum

        return 1
    def update_weights(self,inputs):
        '''update weights and bias for backpropagation'''
        # weight update
        for l in range(0,len(num_neurons)): # for all layers
            for j in range(0,num_neurons[l]): # for all neurons in layer
                if (l == 0): # first hidden layer
                    break
                else:
                    for p in range(0,num_neurons[l-1]): # for all neurons in previous layer
                        self.weights_per_layer[l][j,p] += self.learning_rate * self.local_gradients[l][j] * self.outputs[l-1][p]

                # bias update
                self.biases_per_layer[l][j] += self.learning_rate * self.local_gradients[l][j] * 1
    def num_derivative(self,f,x,delta=1e-6):
        return (f(x+delta)-f(x))/delta


#####blablablablabl#####################################################################################################

np.random.seed(50)
dataset = datasets.load_breast_cancer()
min_max_scaler = preprocessing.MinMaxScaler()
inputs = min_max_scaler.fit_transform(dataset.data)
outputs = dataset.target
np.shape(inputs)
np.shape(outputs)


#clustering utilizando k-means
wcss = []
for i in range(2,8):
    kmeans = KMeans(n_clusters = i, random_state=0).fit(inputs)
    wcss.append(kmeans.inertia_)

print(wcss)
x = np.arange(0,6,1)
plt.plot(x, wcss)

x1, y1 = 2, wcss[0]
x2, y2 = 8, wcss[len(wcss)-1]

distances = []
for i in range(len(wcss)):
    x0 = i+2
    y0 = wcss[i]
    numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
    denominator = sqrt((y2 - y1)**2 + (x2 - x1)**2)
    distances.append(numerator/denominator)

print((distances.index(max(distances))+2))

kmeans = KMeans(n_clusters = 5, random_state=0).fit(inputs)
len(kmeans.cluster_centers_[0])
centers = kmeans.cluster_centers_

train_data1, test_data, train_target1, test_target = train_test_split(inputs, outputs, test_size = 0.2, random_state=50)
train_data, val_data, train_target, val_target = train_test_split(train_data1, train_target1, test_size = 0.25 , random_state=50)

num_outputs = np.size(outputs[0])
num_inputs = np.size(inputs[0])

num_neurons = np.array([6, num_outputs])


num_layers = len(num_neurons)

model = RBFNeuralNetwork(num_inputs, num_outputs, num_neurons, ["gaussian", "linear"], epochs = 5, learning_rate = 0.1, mode = 'sequential')

model.set_centers(centers)
#model.set_weights_interpolation_matrix(inputs, outputs)
sigma = 1
model.fit(train_data,train_target)
predicted = model.predict(val_data)
predicted_class = np.round(predicted)
pontuacao = accuracy_score(val_target,predicted_class)
pontuacao

#otimizando parametros
pontuacao = []
medias = []
value = 0
mean = 0
    for i in np.arange (0.01,0.2, 0.01):
        accuracy = []
        for j in range (3):
            model = RBFNeuralNetwork(num_inputs, num_outputs, num_neurons, ["gaussian", "linear"], epochs = 100, learning_rate = i, mode = 'sequential')
            model.fit(train_data,train_target)
            predicted = model.predict(val_data)
            predicted_class = np.round(predicted)
            pontuacao = accuracy_score(val_target,predicted_class)
            accuracy.append(pontuacao)
        medias.append(np.mean(accuracy))
        if (mean < np.mean(accuracy)):
            mean = np.mean(accuracy)
            value = i

mean
value


model = RBFNeuralNetwork(num_inputs, num_outputs, num_neurons, ["gaussian", "linear"], epochs = 100, learning_rate = 0.03, mode = 'sequential')
model.fit(train_data1,train_target1)
predicted = model.predict(test_data)
predicted_class = np.round(predicted)
accuracy_score(test_target,predicted_class)

x0_sampled = np.arange(-2, 2, 0.1)
x1_sampled = np.arange(-2, 2, 0.1)

x0, x1 = np.meshgrid(x0_sampled, x1_sampled)
z = np.zeros_like(x0)

for i, x0i in enumerate(x0_sampled):
    for j, x1i in enumerate(x1_sampled):
        z[i,j] = model.calc_output([x0i, x1i])

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

ax.set_xlabel("x0")
ax.set_ylabel("x1")
ax.set_zlabel("output")

surf = ax.plot_surface(x0, x1, z, rstride = 1, cstride = 1, cmap = 'viridis', linewidth = 0.1)

x0s = inputs[:,0]
x1s = inputs[:,1]
zs = outputs
ax.scatter(x0s, x1s, zs, marker = 'o', c = 'gray', s = 50, depthshade = 0)

plt.show()
